In [32]:
import time
import os
import requests
import numpy as np
from dataloader import COLDataLoader
import pdb
import pandas as pd

In [44]:
base_ce_path = '../data/bls_ce/msa'
base_zil_path = '../data/zillow/city'
base_tax_path = '../data/state_tax'
data = COLDataLoader(base_zil_path, base_ce_path, base_tax_path)
data.load()
API_KEY = os.environ['GMAP_API_KEY']

/Users/josephshull/Dropbox/code/pygit/cost_of_living_viz/src/dataloader.py:97: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat(list_df, axis=1, ignore_index=False)


In [43]:
class COLGeoUtil():
    def __init__(self, API_KEY = os.environ['GMAP_API_KEY']):
        self.API_KEY = API_KEY
        
    def geocode_one(self, city_state, session1=None, session2=None):
        #Inputs:
        #city_state (array-like): contains str(city) and str(state abbreviation)
        #session (requests.Session()): an instantiated session from the requests,  modeule.
        if isinstance(city_state, (list, tuple, np.ndarray)):
            x = city_state[0] + ', ' + city_state[1]
        else: 
            x = city_state
            
        if session1 is None:
            session1 = session2 = requests
        
        print("getting place id for", x)
        #Fetch ID from Places API
        try:
            places_url = 'https://maps.googleapis.com/maps/api/place/autocomplete/json'
            places_params = { 'input': x, 'key': self.API_KEY, 'types': '(cities)'}
            r = session1.get(places_url, params=places_params)
            results = r.json()
            place_id = results['predictions'][0]['place_id']

            print ("getting geocode for", place_id)
            #Get Lat Lng from Geocoding API
            gc_url = 'https://maps.googleapis.com/maps/api/geocode/json'
            gc_params = {'place_id' : place_id, 'key' : self.API_KEY}
            g = session2.get(gc_url, params=gc_params)
            point = g.json()['results'][0]['geometry']['location']
            lat, lng = point['lat'], point['lng']
            return np.array([lat, lng])
        except:
            return np.array([0, 0])
    
    def get_closest_index(self, lat_lng_series, ref_lat_lng):
        #returns index of closest value in lat_lng_series
        #Inputs:
        #lat_lng_series (np.array): contains numpy arrays of lat,long points
        #ref_lat_lng (array): contains lat,long of reference city
        
        dist = np.linalg.norm(np.vstack(lat_lng_series) - ref_lat_lng, axis=1)
        return np.argmax(dist)
    
    def geocode_dataframe(self, df, city_col, state_col, session1, session2, lat_lng_col='lat_lng'):
        try:
            df.drop(columns=[lat_lng_col])
            df.insert(1,lat_lng_col, 0)
        except:
            df.insert(1,lat_lng_col, 0)
        df[lat_lng_col] = df.loc[:,[city_col, state_col]].apply(self.geocode_one, axis=1, args=(session1,session2,))
        return df
    
    def copy_zil_geocode(self,df_left, df_right, key_cols=['RegionName', 'State'], lat_lng_col='lat_lng'):
        #Prepare to merge only lat,lng
        merge_cols = key_cols
        merge_cols.append(lat_lng_col)
        new_df =  pd.merge(df_left, df_right.loc[:,merge_cols], how='left', on=key_cols)
        #Move lat.lng col to index 3
        feature_cols = list(new_df.columns.values)
        feature_cols.pop(-1)
        feature_cols.insert(lat_lng_col,3)
        
        return new_df.loc[:, feature_cols]
    
    def geocode_ce(self, df):
        cities = df.columns.values
        geocodes = [self.geocode_one(x) for x in cities]
#         df.loc['lat_lng'] = geocodes
        return geocodes
        
        
        
        

In [35]:
gc = COLGeoUtil()

lat_lng = gc.geocode_ce(data.df_ce)

getting place id for Chicago, IL
getting geocode for ChIJ7cv00DwsDogRAMDACa2m4K8
getting place id for  Detroit, MI
getting geocode for ChIJdR3LEAHKJIgR0sS5NU6Gdlc
getting place id for Minneapolis, MN
getting geocode for ChIJvbt3k5Azs1IRB-56L4TJn5M
getting place id for St. Louis, MO
getting geocode for ChIJ-Y7t-qm02IcRW-C7IsrqOb4
getting place id for Washington
D.C.
getting geocode for ChIJW-T2Wt7Gt4kRKl2I1CJFUsI
getting place id for Baltimore
getting geocode for ChIJt4P01q4DyIkRWOcjQqiWSAQ
getting place id for Atlanta, GA
getting geocode for ChIJjQmTaV0E9YgRC2MLmS_e_mY
getting place id for Miami, FL
getting geocode for ChIJEcHIDqKw2YgRZU-t3XHylv8
getting place id for Dallas-
Fort Worth
getting geocode for ChIJrQfILRJuToYRvaxp3fiLr6Q
getting place id for Houston, TX
getting geocode for ChIJAYWNSLS4QIYROwVl894CDco
getting place id for Tampa, FL
getting geocode for ChIJ4dG5s4K3wogRY7SWr4kTX6c
getting place id for New York City, NY
getting geocode for ChIJOwg_06VPwokRYv534QaPC8g
getting pl

In [38]:
ce_geocodes = np.array(lat_lng)

In [52]:
np.save('../data/bls_ce/ce_geocodes', ce_geocodes)

In [55]:
! rm ../data/bls_ce/msa/ce_geocodes.npy

In [56]:
! ls ../data/bls_ce/

ce_geocodes.npy  msa


In [5]:
#GEOCODE DATAFRAME

# start=time.time()
# gc = COLGeoUtil()

# num_rows = 100
# test_df = data.df_zil_own.iloc[:num_rows,:6]
# with requests.Session() as session1:
#     with requests.Session() as session2:
#         df_zil_own = gc.geocode_dataframe(data.df_zil_own, 'RegionName', 'State', session1, session2)
    
# end = time.time()
# print(f"{num_rows} rows took", end-start, "seconds")

# df_zil_own.to_csv('../data/zillow/city/df_zil_own_geocoded.csv')

getting place id for New York, NY
getting geocode for ChIJOwg_06VPwokRYv534QaPC8g
getting place id for New York, NY
getting geocode for ChIJOwg_06VPwokRYv534QaPC8g
getting place id for Los Angeles, CA
getting geocode for ChIJE9on3F3HwoAR9AhGJW_fL-I
getting place id for Chicago, IL
getting geocode for ChIJ7cv00DwsDogRAMDACa2m4K8
getting place id for Houston, TX
getting geocode for ChIJAYWNSLS4QIYROwVl894CDco
getting place id for San Antonio, TX
getting geocode for ChIJrw7QBK9YXIYRvBagEDvhVgg
getting place id for Philadelphia, PA
getting geocode for ChIJ60u11Ni3xokRwVg-jNgU9Yk
getting place id for Phoenix, AZ
getting geocode for ChIJy3mhUO0SK4cRrBtKNfjHaYw
getting place id for Las Vegas, NV
getting geocode for ChIJ0X31pIK3voARo3mz1ebVzDo
getting place id for San Diego, CA
getting geocode for ChIJSx6SrQ9T2YARed8V_f0hOg0
getting place id for Dallas, TX
getting geocode for ChIJS5dFe_cZTIYRj2dH9qSb7Lk
getting place id for San Jose, CA
getting geocode for ChIJ9T_5iuTKj4ARe3GfygqMnbk
getting p

getting geocode for ChIJRcoiUzqXuokRCh56zn8Qmas
getting place id for Glendale, AZ
getting geocode for ChIJN51nLScjK4cRVDuNgwtDFJ8
getting place id for Scottsdale, AZ
getting geocode for ChIJlyx3p9kIK4cRGOaPGBLk0iY
getting place id for Fayetteville, NC
getting geocode for ChIJI0qSQk8Tq4kRmzt2EnWeq6M
getting place id for Boise, ID
getting geocode for ChIJnbRH6XLxrlQRm51nNpuYW5o
getting place id for Irvine, CA
getting geocode for ChIJ40CRaA7d3IAROZpKYHW1eqc
getting place id for Laredo, TX
getting geocode for ChIJJT35p2zAYIYRkXRWSVN6QLQ
getting place id for Sarasota, FL
getting geocode for ChIJK8KyneY3w4gR2RVXHR00NeM
getting place id for Chesapeake, VA
getting geocode for ChIJmW0vgHqluokReQtxkBcbe0w
getting place id for Fort Myers, FL
getting geocode for ChIJKVSoiQFC24gRiiW6ClMIKcY
getting place id for Garland, TX
getting geocode for ChIJdcJf28EDTIYR0DxiOMu9Hwc
getting place id for Gilbert, AZ
getting geocode for ChIJQ65F4A2pK4cRfu_Zf5euU2Q
getting place id for North Las Vegas, NV
getting 

getting place id for Tyler, TX
getting geocode for ChIJ4_mrHgXMSYYRHHB3cX1cKD4
getting place id for Pomona, CA
getting geocode for ChIJIaGfee8tw4ARM6nKAHCUCVI
getting place id for Visalia, CA
getting geocode for ChIJKYfr04LYlIARDkgLAe7jklU
getting place id for Cypress, TX
getting geocode for ChIJGbbrm7nVQIYRx2R8B8goi-I
getting place id for Round Rock, TX
getting geocode for ChIJ5yIt7aTRRIYREEvg7VlyIQc
getting place id for Ann Arbor, MI
getting geocode for ChIJMx9D1A2wPIgR4rXIhkb5Cds
getting place id for Kansas City, KS
getting geocode for ChIJFTXPJ-SMwIcRBR7DcHt3d9w
getting place id for Hollywood, FL
getting geocode for ChIJpzH_x2-p2YgRaLi3db561wc
getting place id for Sunnyvale, CA
getting geocode for ChIJO13QqUW2j4ARosN83Sb7jXY
getting place id for Las Cruces, NM
getting geocode for ChIJaZgGRcca3oYRvTI_8R3QbWM
getting place id for Conroe, TX
getting geocode for ChIJSTttVZEkR4YRDhGtHZsrNeY
getting place id for Lansing, MI
getting geocode for ChIJN4wxfxzAIogRnp0CiSO2eEM
getting place id

getting geocode for ChIJS_yqzmIFVIgRva7NPsC4ofA
getting place id for New Port Richey, FL
getting geocode for ChIJoz7xGdmPwogRtXg0iu2SdQ4
getting place id for Spartanburg, SC
getting geocode for ChIJC0aXphmeV4gRyX7ayjgoi1s
getting place id for Bellingham, WA
getting geocode for ChIJF4dF8i6WhVQR8OA5zamcetU
getting place id for Melbourne, FL
getting geocode for ChIJTZlxRywO3ogRqNIMqVSyzYs
getting place id for Costa Mesa, CA
getting geocode for ChIJeW9X_G_f3IAR2sAD2BKEAe8
getting place id for Sparks, NV
getting geocode for ChIJk2yJCdQ-mYAR-W2FUJpZifE
getting place id for College Station, TX
getting geocode for ChIJozxGGYaERoYRyWq_tk3Hack
getting place id for Bend, OR
getting geocode for ChIJUdLTpf_AuFQRtNEgx6zniBA
getting place id for El Monte, CA
getting geocode for ChIJ79D2Y6TQwoARTTewpY4BD2s
getting place id for Miami Gardens, FL
getting geocode for ChIJZy-MgLuv2YgR1Wvi5YjYme0
getting place id for Chico, CA
getting geocode for ChIJh2Qj_I0fg4AROEU6eY_rjjA
getting place id for Manchester,

getting geocode for ChIJb69GXBgPx4kRAIwwd1DaFoU
getting place id for Federal Way, WA
getting geocode for ChIJ2zYN_RdYkFQRlcAOKBPY2dQ
getting place id for Rio Rancho, NM
getting geocode for ChIJ3S4w_T1lIocRKTJ8_AXuQaw
getting place id for South Gate, CA
getting geocode for ChIJH7dFoxHMwoAR0mrfGYQGQ5E
getting place id for Bryan, TX
getting geocode for ChIJmVMC17yBRoYRcUziFqi05ZU
getting place id for Champaign, IL
getting geocode for ChIJG2mX3o3QDIgRz791tEd49TA
getting place id for Albany, GA
getting geocode for ChIJp3btrUN58ogRSS2xVMRzTfU
getting place id for Portsmouth, VA
getting geocode for ChIJhzeTSraYuokRrfxUY0SqgvU
getting place id for Greece, NY
getting geocode for ChIJ32GNr--w1okRxMpuEgGizXw
getting place id for Idaho Falls, ID
getting geocode for ChIJtRKVc05ZVFMRyUlMCfzJESM
getting place id for Hawthorne, CA
getting geocode for ChIJ8S7MWiu0woARAbM4zB1xwWI
getting place id for Johnson City, TN
getting geocode for ChIJW6NBUdt8WogRy2MFfAwq5jw
getting place id for Woodstock, GA
gett

...

getting geocode for EhtSdXN0b24gV2F5LCBUYWNvbWEsIFdBLCBVU0EiLiosChQKEgn1F15vilSQVBE0AW6mCNqPUhIUChIJZ5VlK-5UkFQRgp6w6wecIWI
getting place id for Avalon, NJ
getting geocode for ChIJVYq0qmylwIkR9qsojY_JR1Y
getting place id for Merrionette Park, IL
getting geocode for ChIJgVKdATU7DogRehtrGY-TSEI
getting place id for Annapolis, CA
getting geocode for ChIJTf5Aa59LgYARqUGCYV4U2Z4
getting place id for Sodus Point, NY
getting geocode for ChIJ7Y-N7UM714kROWR4blvEKpk
getting place id for Harrisville, NH
getting geocode for ChIJ_Tag7_p44YkR9A6aSxlP1eA
getting place id for Eagle, CO
getting geocode for ChIJyS38VlDBQYcR5da4mHUcpMU
getting place id for Rosendale, WI
getting geocode for ChIJeyoKhvb4A4gRl2NJm13OXsI
getting place id for Silverlake, WA
getting geocode for ChIJzX_KpYAPlFQRggz8nHw99oY
getting place id for Granada, MN
getting geocode for ChIJ229IDoyA84cRtUAC8Z3Oo5I
getting place id for Rye, CO
getting geocode for ChIJQ3VloTbxE4cRQ48YJfQxSWs
getting place id for Eastpoint, FL
getting geocod

getting geocode for ChIJM1WMIGz_5IcRgmKd1A1cQWs
getting place id for Ashford, AL
getting geocode for ChIJWXRhyYS_kogR-k53L3oBixI
getting place id for Biltmore Forest, NC
getting geocode for ChIJJVtzQ5DyWYgRLWCIbqAcquk
getting place id for Flemington, PA
getting geocode for ChIJF2vXJ0ryzokRlOD727PvdXE
getting place id for Villano Beach, FL
getting geocode for ChIJ6WT70V0m5IgRt8esXbQUhwI
getting place id for Wesson, MS
getting geocode for ChIJnSoD7MB2KIYRs8YJZzc_DG4
getting place id for Coal Fork, WV
getting geocode for ChIJTxwCIYUqT4gRDVAd3k_Qkfc
getting place id for Belle, WV
getting geocode for ChIJ9RfSZ1jWTogRio8tMOEA3BQ
getting place id for Roaming Shores, OH
getting geocode for ChIJV7p6u7SAMYgRkEm-MJoOGgc
getting place id for Brices Creek, NC
getting geocode for ChIJ3fvXVKTQqIkRcpu0hcXuoQk
getting place id for Concord, AL
getting geocode for EiNDb25jb3JkIEFsbGV5LCBMb3MgQW5nZWxlcywgQ0EsIFVTQSIuKiwKFAoSCXkQl7dTz8KAEXFxBJKdwOf1EhQKEgkT2ifcXcfCgBH0CEYlb98v4g
getting place id for Cerril

getting place id for Tolland, MA
getting geocode for ChIJa-w8LCAR54kR1_emE0J9dBQ
getting place id for Bellevue, IA
getting geocode for ChIJNe6gaozC4ocR3jaTMDxRy8U
getting place id for Oakwood Hills, IL
getting geocode for ChIJSc90hZx0D4gRUp29-M2tFvA
getting place id for Republic, MI
getting geocode for ChIJc9y__Ypbz4cRYq3AOSYrGPc
getting place id for Lake Saint Croix Beach, MN
getting geocode for ChIJ44DewY7g94cROeTBUDEvBG0
getting place id for Kaaawa, HI
getting geocode for ChIJh5-aGv1DAHwR1nrUFML_p2g
getting place id for Forest Lake, IL
getting geocode for ChIJeQX-F2KiD4gRz7YF1CrYOwg
getting place id for Charlemont, MA
getting geocode for ChIJoTn2u23Q4IkRCAO_ZPCb_ZU
getting place id for Egremont, MA
getting geocode for ChIJfVyrzjZk54kRX0JW2Sizjj8
getting place id for Buffalo, IA
getting geocode for ChIJfyYQ-aTP44cROF01qetRy68
getting place id for Ocoee, TN
getting geocode for ChIJHb-f1hczXogROtp40AzQ02c
getting place id for Rio, FL
getting geocode for EiBSaW8gRmxvcmEgUGxhY2UsIERvd25l

getting place id for Boca Grande, FL
getting geocode for ChIJNZe2kFiyxIgRYvcliPZVb34
getting place id for Hague, NY
getting geocode for ChIJpfku7UT-34kRSGt8-JoDZD0
getting place id for Colfax, IA
getting geocode for ChIJ7y8w-Hbl7ocRZfSoQLy3cE4
getting place id for Douglas, MI
getting geocode for ChIJv52tOtH5GYgRIMqG6rtBDwc
getting place id for Sedalia, NC
getting geocode for ChIJdTPpfiYkU4gRtwAoO_Kph3A
getting place id for Forsyth, IL
getting geocode for ChIJV3vTOkOydIgRpynPLXV3Q2Y
getting place id for South Carthage, TN
getting geocode for ChIJvYlNPOGUZogR2neJlOCxuv4
getting place id for Bradenton Beach, FL
getting geocode for ChIJpwaVMUgRw4gRSQam3Aqq6d0
getting place id for Princes Lakes, IN
getting geocode for ChIJ5_3uZhF6a4gRM-R1UfZyTQg
getting place id for Corolla, NC
getting geocode for ChIJdy5r-T4spYkR5QroVcZdu6g
getting place id for Frankston, TX
getting geocode for ChIJedN792JPSIYRxY3jTGMheKs
getting place id for Pine Knoll Shores, NC
getting geocode for ChIJNaH4psmSqIkRKX7ufM

getting place id for Avonmore, PA
getting geocode for ChIJcXOEAyi0NIgRI0Lw5DwEca4
getting place id for North Utica, IL
getting geocode for ChIJdeJyBUlZCYgRpPiIgT2O7gE
getting place id for Crescent Beach, FL
getting geocode for ChIJpa6CyHeZ5ogRrh_g3_saGuY
getting place id for Topsail Beach, NC
getting geocode for ChIJM-AMrhiXqYkRG-yl3YlLtm0
getting place id for June Lake, CA
getting geocode for ChIJq9BN6-ZqloARtP4_vnmMZGU
getting place id for Jonestown, PA
getting geocode for ChIJOzBZpbmryIkRAwFg1Xwc_vk
getting place id for Rushford, MN
getting geocode for ChIJ5TB7Qs59-YcRmIAbh1UQoBo
getting place id for Tiki Island, TX
getting geocode for ChIJCeCmTgiEP4YRYHSxSexcgC4
getting place id for Wellston, OK
getting geocode for ChIJt1tSngzVs4cRpRenom4WiFw
getting place id for Bridgeport, NY
getting geocode for ChIJ-SFHDLWW2YkR5dF4OBuCFfI
getting place id for Rose Hill, NC
getting geocode for ChIJaQ70HiTaq4kR6WahQaIsEJY
getting place id for Belleville, WI
getting geocode for ChIJ14p3sIDHB4gR3fLP

In [31]:
data.df_ce.loc['lat_lng'] = lat_lng

ValueError: setting an array element with a sequence.

In [8]:
data.df_ce

,"Chicago, IL","Detroit, MI",Minneapolis St. Paul,"St. Louis, MO",Washington D.C.,Baltimore,"Atlanta, GA","Miami, FL",Dallas- Fort Worth,"Houston, TX",...,"Philadelphia, PA","Boston, MA",Los Angeles,San Francisco,San Diego,Seattle,Phoenix,Denver,Honolulu,Anchorage
Adults 65 and older,0.4,0.4,0.3,0.4,0.3,0.4,0.3,0.4,0.2,0.3,...,0.4,0.4,0.3,0.4,0.3,0.3,0.4,0.3,0.5,0.3
Age of reference person,49.9,52.9,48.9,50.4,49.1,52.1,47.7,54.2,46.9,46.8,...,53.3,49.6,49.8,49.6,49.2,47.9,50.0,46.7,53.0,49.1
Alcoholic beverages,509.0,658.0,754.0,684.0,662.0,724.0,385.0,487.0,576.0,650.0,...,492.0,823.0,620.0,875.0,1112.0,986.0,485.0,771.0,658.0,788.0
Apparel and services,1987.0,2102.0,1967.0,1567.0,2668.0,1914.0,1874.0,2602.0,2353.0,2606.0,...,2125.0,2134.0,2150.0,1878.0,2563.0,3884.0,1635.0,2387.0,1667.0,1648.0
Average annual expenditures,60582.0,60183.0,70487.0,62394.0,79921.0,69126.0,62250.0,59947.0,63207.0,67304.0,...,65436.0,74316.0,66971.0,79291.0,79585.0,83790.0,62072.0,74088.0,65099.0,71606.0
Average number in consumer unit:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash contributions,1685.0,1526.0,4269.0,4201.0,2426.0,1502.0,1945.0,933.0,1517.0,2093.0,...,1560.0,2665.0,1522.0,2412.0,2038.0,3502.0,1452.0,2397.0,3512.0,1867.0
Cereals and bakery products,539.0,563.0,596.0,505.0,556.0,551.0,443.0,524.0,393.0,568.0,...,696.0,712.0,549.0,599.0,638.0,754.0,513.0,622.0,511.0,655.0
Children under 18,0.6,0.5,0.5,0.5,0.6,0.6,0.8,0.4,0.6,0.8,...,0.5,0.5,0.7,0.5,0.7,0.5,0.7,0.6,0.6,0.6
Consumer unit characteristics:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
os.path.join(base_ce_path, 'join')

'../data/bls_ce/msa/join'